# 音声データ分析 - Earnings Call Q1 FY2025

Snowflake Cortex AI関数を使用して音声データを分析します。

**分析内容:**
1. AI_TRANSCRIBE - 音声の文字起こし
2. AI_CLASSIFY - コンテンツ分類
3. AI_AGG - サマリー生成
4. AI_TRANSLATE - 英語から日本語への翻訳
5. AI_COMPLETE - ビジネスインサイト抽出（Structured Output）

**使用データ:** `@ai_discover.public.raw_stage/audio/EARNINGS_Q1_FY2025.mp3`

In [ ]:
%%sql -r dataframe_1
USE SCHEMA ai_discover.public;

---
## 1. AI_TRANSCRIBE - 音声文字起こし

`AI_TRANSCRIBE`は音声ファイルをテキストに変換するCortex AI関数です。

**特徴:**
- MP3、WAV、FLAC等の音声形式をサポート
- 複数言語の自動検出に対応
- タイムスタンプ付きの文字起こしも可能

**構文:**
```sql
AI_TRANSCRIBE(TO_FILE('@stage/audio_file.mp3'))
```

In [ ]:
%%sql -r transcribe_result
CREATE OR REPLACE TABLE AUDIO_SAMPLE AS
SELECT 
    '@ai_discover.public.raw_stage/audio/EARNINGS_Q1_FY2025.mp3' AS audio_file,
    AI_TRANSCRIBE(
        TO_FILE('@ai_discover.public.raw_stage/audio/EARNINGS_Q1_FY2025.mp3')
    ):text::STRING AS transcript_text

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()
df = session.sql('SELECT TRANSCRIPT_TEXT FROM AUDIO_SAMPLE').to_pandas()
text = df['TRANSCRIPT_TEXT'].iloc[0]
print(f'文字数: {len(text):,}文字')
print(f'\n--- 文字起こし結果（最初の2000文字）---\n{text[:2000]}...')

---
## 2. AI_CLASSIFY - コンテンツ分類

`AI_CLASSIFY`はテキストを指定したカテゴリに分類するCortex AI関数です。

**特徴:**
- 任意のカテゴリリストを指定可能
- 確信度スコア付きで結果を返却
- マルチラベル分類にも対応

**構文:**
```sql
AI_CLASSIFY(text, ['Category1', 'Category2', ...])
```

In [ ]:
%%sql -r classify_result
CREATE OR REPLACE TABLE AUDIO_SAMPLE_2 AS
SELECT 
    *,
    AI_CLASSIFY(
        TRANSCRIPT_TEXT,
        [
            'Financial Results',
            'Product Announcement',
            'Market Strategy',
            'Risk Disclosure',
            'Future Guidance',
            'Partnership News',
            'Technology Innovation'
        ]
    ):labels[0]::STRING AS content_category
FROM AUDIO_SAMPLE

In [ ]:
df = session.sql('SELECT CONTENT_CATEGORY FROM AUDIO_SAMPLE_2').to_pandas()
print(f'コンテンツ分類: {df["CONTENT_CATEGORY"].iloc[0]}')

---
## 3. AI_AGG - テキスト要約

`AI_AGG`は複数行のテキストを集約・要約するCortex AI関数です。

**特徴:**
- GROUP BYと組み合わせて複数レコードを要約
- カスタムプロンプトで要約の形式を指定可能
- 長文を指定した文字数に圧縮

**構文:**
```sql
AI_AGG(text_column, 'Summarize in 400 words') ... GROUP BY ...
```

In [ ]:
%%sql -r sentiment_result
CREATE OR REPLACE TABLE AUDIO_SAMPLE_3 AS
SELECT 
    *,
    AI_AGG(
        TRANSCRIPT_TEXT, 
        'Summarize given text within 400 words'
    ) AS summary_result
FROM AUDIO_SAMPLE_2
GROUP BY ALL

In [ ]:
df = session.sql('SELECT SUMMARY_RESULT FROM AUDIO_SAMPLE_3').to_pandas()
print(f'--- サマリー ---\n{df["SUMMARY_RESULT"].iloc[0]}')

---
## 4. AI_TRANSLATE - 多言語翻訳

`AI_TRANSLATE`はテキストを指定した言語に翻訳するCortex AI関数です。

**特徴:**
- 100以上の言語をサポート
- ISO 639-1言語コードを使用
- 自然で高品質な翻訳を生成

**構文:**
```sql
AI_TRANSLATE(text, 'source_lang', 'target_lang')
-- 例: AI_TRANSLATE(text, 'en', 'ja') -- 英語から日本語へ
```

In [ ]:
%%sql -r translate_result
CREATE OR REPLACE TABLE AUDIO_SAMPLE_4 AS
SELECT 
    *,
    AI_TRANSLATE(
        summary_result,
        'en',
        'ja'
    ) AS summary_japanese_translation
FROM AUDIO_SAMPLE_3

In [ ]:
df = session.sql('SELECT SUMMARY_JAPANESE_TRANSLATION FROM AUDIO_SAMPLE_4').to_pandas()
print(f'--- 日本語翻訳 ---\n{df["SUMMARY_JAPANESE_TRANSLATION"].iloc[0]}')

---
## 5. AI_COMPLETE - ビジネスインサイト抽出（Structured Output）

`AI_COMPLETE`は高度なLLM推論を実行するCortex AI関数です。

**特徴:**
- Claude、Mistral、Llama等の複数モデルをサポート
- **Structured Output**: JSON Schemaで出力形式を厳密に定義可能
- 複雑な分析タスクに最適

**Structured Outputの構文:**
```sql
AI_COMPLETE(
    model => 'claude-sonnet-4-5',
    prompt => 'Your prompt here',
    response_format => {
        'type': 'json',
        'schema': {
            'type': 'object',
            'properties': {...},
            'required': [...]
        }
    }
)
```

**今回の分析項目:**
- 注力製品・サービス
- 成長ドライバー
- リスク要因
- 競合優位性
- 投資家へのメッセージ

In [ ]:
%%sql -r analysis_result
CREATE OR REPLACE TABLE AUDIO_SAMPLE_5 AS
SELECT 
    *,
    AI_COMPLETE(
        model => 'claude-sonnet-4-5',
        prompt => 'あなたは投資アナリストです。以下のEarnings Callの内容を分析し、日本語で回答してください。\n\n## Earnings Call内容:\n' || t.TRANSCRIPT_TEXT,
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'focus_products': {'type': 'string', 'description': '今後注力する製品やサービス'},
                    'growth_drivers': {'type': 'string', 'description': '売上成長を牽引する要因'},
                    'risk_factors': {'type': 'string', 'description': '経営上のリスクや課題'},
                    'competitive_advantage': {'type': 'string', 'description': '競合他社と比較した強み'},
                    'investor_message': {'type': 'string', 'description': '経営陣が最も伝えたいこと'}
                },
                'required': ['focus_products', 'growth_drivers', 'risk_factors', 'competitive_advantage', 'investor_message']
            }
        }
    ) AS business_insights
FROM AUDIO_SAMPLE_4 t

In [ ]:
import json
df = session.sql('SELECT BUSINESS_INSIGHTS FROM AUDIO_SAMPLE_5').to_pandas()
insights = df['BUSINESS_INSIGHTS'].iloc[0]
if isinstance(insights, str):
    insights = json.loads(insights)

labels = {
    'focus_products': '注力製品・サービス',
    'growth_drivers': '成長ドライバー',
    'risk_factors': 'リスク要因',
    'competitive_advantage': '競合優位性',
    'investor_message': '投資家へのメッセージ'
}
print('=' * 60)
print('ビジネスインサイト分析結果')
print('=' * 60)
for key, label in labels.items():
    if key in insights:
        print(f'\n【{label}】\n{insights[key]}')

---
## まとめ

### 使用したCortex AI関数

| 関数 | 用途 | 出力 |
|------|------|------|
| `AI_TRANSCRIBE` | 音声→テキスト変換 | 文字起こしテキスト |
| `AI_CLASSIFY` | テキスト分類 | カテゴリラベル |
| `AI_AGG` | テキスト要約 | 要約テキスト |
| `AI_TRANSLATE` | 多言語翻訳 | 翻訳テキスト |
| `AI_COMPLETE` | LLM推論 | 構造化JSON |

### 分析フロー

```
音声ファイル(.mp3)
    ↓ AI_TRANSCRIBE
テキスト
    ↓ AI_CLASSIFY
分類済みテキスト
    ↓ AI_AGG
要約テキスト
    ↓ AI_TRANSLATE
日本語要約
    ↓ AI_COMPLETE (Structured Output)
ビジネスインサイト（JSON）
```

### ポイント

1. **Structured Output**: `response_format`パラメータでJSON Schemaを指定することで、LLMの出力形式を厳密に制御
2. **パイプライン処理**: 各ステップの結果をテーブルに保存し、次のステップで参照
3. **非構造化データの活用**: 音声という非構造化データから構造化されたビジネスインサイトを抽出

In [ ]:
print('=' * 60)
print('Earnings Call 分析完了')
print('=' * 60)
df = session.sql('''
    SELECT 
        AUDIO_FILE,
        CONTENT_CATEGORY,
        LENGTH(TRANSCRIPT_TEXT) AS transcript_length,
        LENGTH(SUMMARY_RESULT) AS summary_length
    FROM AUDIO_SAMPLE_5
''').to_pandas()
print(f"\n音声ファイル: {df['AUDIO_FILE'].iloc[0]}")
print(f"コンテンツ分類: {df['CONTENT_CATEGORY'].iloc[0]}")
print(f"文字起こし文字数: {df['TRANSCRIPT_LENGTH'].iloc[0]:,}文字")
print(f"要約文字数: {df['SUMMARY_LENGTH'].iloc[0]:,}文字")
print(f"\n生成テーブル:")
print("  - AUDIO_SAMPLE (文字起こし)")
print("  - AUDIO_SAMPLE_2 (分類)")
print("  - AUDIO_SAMPLE_3 (要約)")
print("  - AUDIO_SAMPLE_4 (翻訳)")
print("  - AUDIO_SAMPLE_5 (インサイト)")